In [1]:
## Gerekli Kütüphaneler
import pandas as pd
import numpy as np

In [28]:
## Anket Verisi
data  =  pd.read_excel(r'https://raw.githubusercontent.com/aiasci/HHBA/main/fert_cd_2019.excel')

In [3]:
# TÜİK Ağırlıklandırması
ağırlık = []
for i in range(0, len(data)):
    if data['FERTNO'][i] == 1:
        ağırlık.append(data['FERTNO'][i])
    if data['FERTNO'][i] != 1:
        if data['YAS'][i] < 14:
            ağırlık.append(0.3)
    if data['FERTNO'][i] != 1:
        if data['YAS'][i] >= 14:
            ağırlık.append(0.5)
# Veriye Ağırlıkların Eklenmesi
data['ağırlık'] = ağırlık

In [4]:
# Gelirlerin HAnelere göre Toplulaştırılması
hanehalkı_gelirler = data[['BIRIMNO', 'GELIR_TOPLAM','ağırlık']].groupby('BIRIMNO').sum() 
hanehalkı_gelirler = hanehalkı_gelirler.reset_index()
# Hane Başına Eşdeğer Fert Gelirlerinin Hesaplanması
hanehalkı_gelirler['Ortalama'] = hanehalkı_gelirler['GELIR_TOPLAM']/hanehalkı_gelirler['ağırlık']
# Eşdeğer Fert Gelirlerine göre Hanelerin Sıralanması 
hanehalkı_gelirler.sort_values('Ortalama', inplace=True)

In [5]:
## Eğer Haneler Yalnızca Toplam Gelirlerine göre sıralanmak isteniyorsa Üstteki Blok yerine
## Yalnızca bu kod çalıştırılabilir
#hanehalkı_gelirler.sort_values('GELIR_TOPLAM', inplace=True)

In [6]:
# İndex'in düzeltilmesi
hanehalkı_gelirler =hanehalkı_gelirler.reset_index(drop=True)

In [7]:
# TÜİK Metodolojisi ile %20'lik gelir gruplarının oluşturulması
hh_1 = hanehalkı_gelirler[:2305].reset_index(drop=True)
hh_1['Gelir Grubu'] = 1
hh_2 = hanehalkı_gelirler[2305:4609].reset_index(drop=True)
hh_2['Gelir Grubu'] = 2
hh_3 = hanehalkı_gelirler[4609:6913].reset_index(drop=True)
hh_3['Gelir Grubu'] = 3
hh_4 = hanehalkı_gelirler[6913:9217].reset_index(drop=True)
hh_4['Gelir Grubu'] = 4
hh_5 = hanehalkı_gelirler[9217:11521].reset_index(drop=True)
hh_5['Gelir Grubu'] = 5
# Grupların Toplulaştırılması
gruplar_main = pd.concat([hh_1, hh_2, hh_3, hh_4, hh_5])

In [8]:
## Alt gelir gruplarını oluşturan hanelerin listelenmesi
#a= pd.DataFrame()
#for i in range(1,6):
#    vars()['data_hh_'+str(i)] = pd.DataFrame()
#    for j in range(0,len(vars()['hh_'+str(i)])):
#        a= data[data['BIRIMNO'] == vars()['hh_'+str(i)]['BIRIMNO'][j]]
#        vars()['data_hh_'+str(i)] = vars()['data_hh_'+str(i)].append(a)
## Alt Gelir Gruplarının Kaydedilmesi
#for i in range(1,6):
#    vars()['data_hh_'+str(i)].to_excel(f'grup[{i}.xlsx',index=False) 

In [ ]:
# Hanelerin tüketim harcamaları verisi
har = pd.read_csv(r'https://raw.githubusercontent.com/aiasci/HHBA/main/tuketim_cd_2019.csv')
# Tüketim maddelerinin HS-5 Koduna göre isimlerini belirten dosya
madde = pd.read_excel(r'https://github.com/aiasci/HHBA/blob/main/hbs_madde_tanim.xlsx?raw=true', sheet_name='pro')

In [ ]:
# Tüketim Harcamaları ile Ürün İsimlerinin birleştirilmesi
har_main = har.merge(madde, on = 'HBS_KOD5', how = 'inner')
# Hanelerin Tüketimleri ile Hane kodlarının gelir gruplarıyla birlikte birleştirilmesi
df1 = har_main.merge(gruplar_main, on = 'BIRIMNO', how = 'inner')
# Her Gelir grubunun gelirlerinin toplanması
gruplar_genel = gruplar_main[['GELIR_TOPLAM', 'Gelir Grubu']].groupby('Gelir Grubu').sum().reset_index()
# Ana verinin gerekli ögelerle daraltılması
df1 = df1[['BIRIMNO', 'HBS_KOD5', 'DEGER_D', 'tür', 'Grup adı', 'GELIR_TOPLAM', 'Gelir Grubu']]
# Gelir gruplarına göre Ürün tüketimleri
ürün_gelirgrubu = df1[['HBS_KOD5', 'DEGER_D', 'Gelir Grubu']].groupby(['HBS_KOD5', 'Gelir Grubu']).sum().reset_index()
# Gelir gruplarına göre Ürün tüketimleriyle gelir gruplarının toplam gelirlerinin birleştirilmesi
data_son = ürün_gelirgrubu.merge(gruplar_genel, on = 'Gelir Grubu', how = 'inner')
# Ürün isimlerinin birleştirilmesi
data_son = data_son.merge(madde[['HBS_KOD5', 'tür', 'Grup adı']], on = 'HBS_KOD5', how = 'inner')
# Her bir ürünün gelir gruplarının sepetinde ne kadar yer tuttuğu
data_son['oran'] = data_son['DEGER_D']/(data_son['GELIR_TOPLAM']/12)
data_son

In [ ]:
# Gelir Gruplarına Göre Toplam Gıda Tüketiminin Gelir Grubu Ortalama Gelirine Oranı
#data_son[(data_son['Gelir Grubu'] == 1) & (data_son['tür'] == 'GIDA')]['oran'].sum()
# Ürün bazlı gıda tüketiminin gelir grubu ortalama gelirine oranı (HBS_KOD5'e göre)
#data_son[(data_son['Gelir Grubu'] == 1) & (data_son['tür'] == 'GIDA') & (data_son['HBS_KOD5'] == 1111)]['oran'].sum()
# Ürün bazlı gıda tüketiminin gelir grubu ortalama gelirine oranı (Grup Adına göre)
#data_son[(data_son['Gelir Grubu'] == 1) & (data_son['tür'] == 'GIDA') & (data_son['Grup adı'] == 'PİRİNÇ')]['oran'].sum()